In [ ]:
!pip install openai
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install tiktoken

In [142]:
import os
import sys
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import openai

from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from gensim.models.phrases import Phrases, Phraser

In [41]:
import cv2
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import pandas as pd
import re
import subprocess
import random

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [6]:
os.environ["OPENAI_API_KEY"] = ""

In [7]:
# loader = TextLoader("/content/drive/MyDrive/Colab Notebooks/script/abstracts.csv") # Use this line if you only need data.txt
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/script/International-GCSE-Double-Award-Science-Student-Book-sample.pdf")
# loader = DirectoryLoader("data/")
index = VectorstoreIndexCreator().from_loaders([loader])

In [8]:
chain = ConversationalRetrievalChain.from_llm(
  llm=ChatOpenAI(model="gpt-3.5-turbo-16k"),
  # llm=ChatOpenAI(model="gpt-4"),
  retriever=index.vectorstore.as_retriever(search_kwargs={"k": 1}),
)

In [152]:
# Run from here to clear the history.
chat_history = []

In [ ]:
# ------ Enter query here ------
query = "generate topics from the document"
result = chain({"question": query, "chat_history": chat_history})
chat_history.append((query, result['answer']))
response = (result['answer'])

# Function to find and display relevant images based on extracted text
def display_images_for_text(extracted_text, image_folder):
    # Define a list of keywords and corresponding image filenames
    keywords_images = {
        "smoking": "/content/drive/MyDrive/Colab Notebooks/image_text_extraction_/extracted_images/smoking.png",
        "illnesses": "/content/drive/MyDrive/Colab Notebooks/image_text_extraction_/extracted_images/illnesses.png",
        # Add more keywords and image filenames as needed
    }

    matched_keywords = []
    for keyword, image_filename in keywords_images.items():
      if re.search(keyword, extracted_text, re.IGNORECASE):
        matched_keywords.append((keyword, image_filename))

    if matched_keywords:
        # print("Relevant images found for keywords:")
        for keyword, image_filename in matched_keywords:
            image_path = os.path.join(image_folder, image_filename)
            if os.path.exists(image_path):
                print(f"Keyword: {keyword}")
                return keyword
                # # Use an external viewer to display the image (e.g., on Windows)
                # subprocess.run(["start", "explorer", image_path], shell=True)
            else:
                print(f"Image not found for keyword: {keyword}")
    else:
        print("No relevant images found for the extracted text.")

# Response text and image file path
image_folder_path =  "/content/drive/MyDrive/Colab Notebooks/image_text_extraction_/extracted_images/"
query_text = response

#  Display relevant images based on extracted text
print(f"GPT Response: {response}")
relevant_image = str(display_images_for_text(query_text, image_folder_path))
if relevant_image == "None":
  pass
else:
  img = Image.open(f"/content/drive/MyDrive/Colab Notebooks/image_text_extraction_/extracted_images/{relevant_image}.png")
  display(img)